# Evaluation

In [1]:
from pathlib import Path
from evaluation_utils import print_statistics, get_document_text
import pandas as pd
import json
import numpy as np

data_dir = Path('data')
annotation_file = data_dir / 'emc-dcc_ann.json'
dcc_dir = data_dir / 'EMCDutchClinicalCorpus'
result_dir = Path('results')
bilstm_result_file = result_dir / 'bilstm_predictions.csv.gz'
rule_based_result_file = result_dir / 'rule-based_predictions.csv.gz'
merged_result_file = result_dir / 'merged_results.csv.gz'

# Load annotated data
with open(annotation_file) as f:
    annotations = json.load(f)
    
# Load biLSTM results
bilstm_annotations = pd.read_csv(bilstm_result_file)

# Load rule based results
ruled_based_annotations = pd.read_csv(rule_based_result_file)

In [2]:
bilstm_annotations.shape

(12551, 2)

In [3]:
# Load annotated data
result = []
for document in annotations['projects'][0]['documents']:
    document_name = document['name']
    text = document['text']

    for annotation in document['annotations']:

        # Extract data
        start_char = annotation['start']
        end_char = annotation['end']
        negation_value = annotation['meta_anns']['Negation']['value']

        # Create custom ID
        entity_id = f'{document_name}_{start_char}_{end_char}'
        result.append([entity_id, negation_value])
results = pd.DataFrame(result, columns=['entity_id', 'label'])

## Merge annotations from different methods

In [4]:
results = pd.merge(left=results, right = bilstm_annotations, left_on='entity_id', right_on='entity_id')

In [5]:
results = pd.merge(left=results, right = ruled_based_annotations, left_on='entity_id', right_on='entity_id')
results.drop(['annotation'], axis=1, inplace=True)

In [6]:
results.head(5)

,entity_id,label,bilstm,rule_based
0,DL1111_32_46,not negated,not negated,not negated
1,DL1111_272_280,not negated,not negated,not negated
2,DL1111_363_377,not negated,not negated,not negated
3,DL1112_22_28,negated,negated,negated
4,DL1113_59_67,not negated,not negated,not negated


## Results biLSTM

In [7]:
print_statistics(results, 'bilstm')

tp: 1737
tn: 10766
fp: 25
fn: 23
recall: 0.99
precision: 0.99
specificity: 1.0
accuracy: 1.0
f1: 0.99


## Results Rule Based

In [8]:
print_statistics(results, 'rule_based')

tp: 1569
tn: 10455
fp: 336
fn: 191
recall: 0.89
precision: 0.82
specificity: 0.97
accuracy: 0.96
f1: 0.86


## Save merged results

In [9]:
results.to_csv(merged_result_file, index=False, compression='gzip')